## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [ ]:
# answer goes here

raw_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')

raw_data

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,0.367,0.2,1.1,0.217,1.3,3.0,0.423,0.396,0.5,0.6,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,0.339,1.4,4.4,0.325,0.9,2.5,0.363,0.442,1.2,1.4,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,0.529,0.0,0.1,0.100,3.2,5.8,0.539,0.530,2.4,3.2,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [ ]:
# answer goes here
data = data[data['GS'] >= 20]
data = data[data['MP'] >= 10]

In [ ]:
feats_list = ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV']

feats_df = data[feats_list]
feats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 3 to 2139
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PTS     804 non-null    float64
 1   TRB     804 non-null    float64
 2   AST     804 non-null    float64
 3   STL     804 non-null    float64
 4   BLK     804 non-null    float64
 5   TOV     804 non-null    float64
dtypes: float64(6)
memory usage: 44.0 KB


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [ ]:
scale = StandardScaler()
scaled = scale.fit_transform(feats_df)
print(scaled)

[[ 0.11463777  1.6129094  -0.61781554  1.3932204   0.82400066  0.02351072]
 [-0.7449851   0.76862715 -0.33088552 -0.05027084  0.42016248 -0.21883054]
 [ 1.38687962  1.49778    -0.23524218 -1.01259833  1.42975792  0.14468135]
 ...
 [-0.2635963   0.42323896 -0.71345889 -0.53143459  0.82400066 -1.06702496]
 [ 0.32094726  1.42102707 -0.5221722   1.3932204  -0.18559478  0.26585198]
 [-0.77937002  0.34648603 -0.90474557 -0.29085271  0.62208157 -0.94585433]]


In [ ]:
kmeans = KMeans(n_clusters= 3).fit(scaled)
clusters = kmeans.cluster_centers_
unscaled = scale.inverse_transform(clusters)
print(unscaled)

[[13.73535912  8.60552486  1.91491713  0.77734807  1.23038674  1.66850829]
 [10.50540541  3.94977477  2.06373874  0.80472973  0.37432432  1.23513514]
 [19.49162011  5.29385475  5.93407821  1.35418994  0.48603352  2.79776536]]


In [ ]:
kmeans = KMeans(n_clusters= 5).fit(scaled)
clusters = kmeans.cluster_centers_
unscaled = scale.inverse_transform(clusters)
print(unscaled)

[[18.73030303 10.57727273  2.6030303   0.97272727  1.69242424  2.31515152]
 [ 7.75515464  3.47835052  1.40979381  0.60824742  0.35618557  0.90670103]
 [19.76083916  5.23496503  6.29090909  1.3972028   0.45664336  2.88181818]
 [11.48518519  7.2382716   1.66049383  0.68395062  0.92654321  1.38333333]
 [13.44100418  4.03682008  2.97573222  1.03598326  0.3334728   1.61631799]]


In [ ]:
archetype= kmeans.fit_predict(scaled)
archetype.shape

(804,)

In [ ]:
clusters.shape

(5, 6)

In [ ]:
feats_df.shape

(804, 6)

In [ ]:
feats_df['archetype'] = archetype 

feats_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PTS,TRB,AST,STL,BLK,TOV,archetype
3,13.9,9.5,1.6,1.5,1.0,1.7,0
4,8.9,7.3,2.2,0.9,0.8,1.5,0
7,21.3,9.2,2.4,0.5,1.3,1.8,3
10,10.9,8.4,1.4,0.5,1.5,1.3,0
12,9.4,7.5,1.3,0.8,0.4,0.9,0
...,...,...,...,...,...,...,...
2122,14.1,2.9,5.8,0.9,0.2,2.3,4
2125,15.3,2.5,2.5,0.9,0.3,1.6,4
2126,11.7,6.4,1.4,0.7,1.0,0.8,0
2138,15.1,9.0,1.8,1.5,0.5,1.9,4


Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [ ]:
# answer goes here
GMM = GaussianMixture(n_components= 5).fit(scaled)
probabilities = GMM.predict_proba(scaled)
clusters = GMM.fit_predict(scaled)
#unscaled = scale.inverse_transform(clusters)
#print(unscaled)

In [ ]:
print(probabilities)

[[2.67682020e-09 5.30569210e-01 1.74067750e-01 1.15921512e-22
  2.95363038e-01]
 [1.67158556e-03 9.46311546e-01 3.47150484e-02 3.28770211e-15
  1.73018197e-02]
 [1.41660721e-15 2.34843450e-01 7.64939719e-01 4.86149696e-33
  2.16830759e-04]
 ...
 [3.71326871e-05 9.85663621e-01 1.33311969e-02 1.21908121e-16
  9.68049298e-04]
 [3.07060543e-09 8.46569296e-01 1.68300087e-02 9.13336897e-17
  1.36600692e-01]
 [1.87198550e-02 9.76003165e-01 2.87082875e-03 2.94972424e-14
  2.40615117e-03]]


In [ ]:
probabilities.shape

(804, 5)

In [ ]:
clusters

array([4, 4, 3, 3, 4, 4, 2, 1, 1, 1, 3, 1, 4, 4, 4, 4, 1, 0, 3, 0, 0, 1,
       4, 1, 4, 4, 0, 4, 0, 4, 1, 0, 4, 0, 2, 4, 3, 0, 4, 0, 3, 1, 1, 3,
       2, 0, 0, 0, 1, 3, 0, 1, 3, 0, 1, 3, 0, 1, 3, 0, 2, 0, 0, 0, 0, 4,
       1, 2, 4, 3, 4, 2, 0, 4, 3, 3, 4, 0, 0, 3, 0, 2, 4, 3, 1, 2, 0, 4,
       4, 4, 0, 0, 4, 4, 2, 4, 0, 1, 3, 1, 3, 1, 4, 1, 0, 3, 4, 1, 2, 4,
       4, 2, 0, 1, 3, 3, 0, 4, 0, 4, 4, 1, 0, 0, 2, 1, 1, 0, 3, 3, 3, 1,
       4, 1, 1, 3, 0, 4, 3, 4, 4, 1, 4, 4, 1, 1, 4, 0, 3, 0, 4, 1, 0, 4,
       2, 1, 0, 4, 4, 0, 3, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 4, 2, 2, 1, 4,
       1, 0, 4, 3, 3, 4, 3, 3, 1, 0, 3, 1, 1, 2, 4, 4, 2, 4, 3, 4, 4, 0,
       1, 4, 1, 0, 0, 0, 3, 3, 0, 4, 4, 0, 3, 4, 0, 4, 1, 2, 4, 3, 1, 0,
       4, 1, 4, 0, 0, 0, 2, 1, 4, 1, 1, 1, 4, 4, 4, 2, 1, 4, 3, 4, 0, 3,
       0, 0, 0, 1, 2, 4, 4, 0, 0, 1, 3, 0, 1, 1, 1, 3, 2, 3, 2, 0, 1, 0,
       1, 1, 1, 4, 3, 3, 2, 0, 3, 4, 4, 0, 1, 0, 2, 3, 3, 1, 2, 0, 4, 4,
       4, 0, 0, 4, 4, 2, 4, 4, 4, 3, 3, 3, 0, 1, 3,

In [ ]:
#feats_df['probabilities'] = probabilities 
feats_df['clusters'] = clusters

feats_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,PTS,TRB,AST,STL,BLK,TOV,archetype,clusters
3,13.9,9.5,1.6,1.5,1.0,1.7,0,4
4,8.9,7.3,2.2,0.9,0.8,1.5,0,4
7,21.3,9.2,2.4,0.5,1.3,1.8,3,3
10,10.9,8.4,1.4,0.5,1.5,1.3,0,3
12,9.4,7.5,1.3,0.8,0.4,0.9,0,4
...,...,...,...,...,...,...,...,...
2122,14.1,2.9,5.8,0.9,0.2,2.3,4,1
2125,15.3,2.5,2.5,0.9,0.3,1.6,4,4
2126,11.7,6.4,1.4,0.7,1.0,0.8,0,0
2138,15.1,9.0,1.8,1.5,0.5,1.9,4,4


Predict the likelihood of each season belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those seasons.

In [ ]:
# answer goes here



